In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import random

import accelerate
import torch
import transformers

from src._shared import (
    apply_lora_to_model,
    freeze_base_models,
    load_clip_model,
    load_config,
    load_tokenizers,
    prepare_dataset,
    save_model_and_logs,
    setup_environment,
    setup_trainer,
    train_model,
)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

train_config = load_config()
model_name_identifier, device, report_to, run, USE_WANDB, SEED = setup_environment(train_config)

accelerate.utils.set_seed(SEED + 1)
transformers.set_seed(SEED + 2)
torch.manual_seed(SEED + 3)
random.seed(SEED + 4)

In [ ]:
tokenizer_plm, tokenizer_llm = load_tokenizers(train_config)
dataset = prepare_dataset(train_config, tokenizer_plm, tokenizer_llm)

In [ ]:
model = load_clip_model(train_config, device)

In [ ]:
print(model.logit_scale.scale)
print(model.protein_projection.weight)
print(model.text_projection.weight)

In [5]:
model.load_projections_from_safetensors('../tmp/models/protT5-CLIP-2025-01-17-21-47-10-0/')

In [ ]:
print(model.logit_scale.scale)
print(model.protein_projection.weight)
print(model.text_projection.weight)